In [0]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [3]:
# Google Colab stuff
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
# create a dataframe using texts and lables
trainDF = train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/data/training_data.csv')
trainDF.head()

,essay_id,text,ideas_content,organization,voice,word_choice,sentence_fluency,conventions,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,A long time ago when I was in third grade I h...,4,4,4,4,4,3,NaN,NaN,NaN
1,2,Softball has to be one of the single most gre...,5,4,5,4,4,4,NaN,NaN,NaN
2,3,"Some people like making people laugh, I love ...",3,3,3,3,3,3,NaN,NaN,NaN
3,4,"""LAUGHTER"" @CAPS1 I hang out with my friends...",3,3,3,3,2,2,NaN,NaN,NaN
4,5,Well ima tell a story about the time i got @CA...,3,2,3,3,2,2,NaN,NaN,NaN


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
class Dataset:
    def __init__(self, path):

        self.df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/training_data.csv")
        self.data = self.df.to_numpy()



        self.stopwords = set(stopwords.words('english'))
        self.essay_id = self.data[:,0]
        self.text = self.data[:,1]
        self.scores = self.data[:,2:8]
        self.new_data = []
        self.vocab = set()
        self.word_to_id = None

    def preprocess(self):
        for i in range(len(self.essay_id)):
            text = self.text[i].lower()
            text = " ".join([word for word in text.split() if '@' not in word])
            text = word_tokenize(text)
            text = [word for word in text if word not in self.stopwords]
            self.text[i] = text 

    def create_vocab(self):
        for line in self.text:
            for word in line:
                self.vocab.add(word)

        self.vocab = sorted(list(self.vocab))
        self.word_to_id = {word:i for i, word in enumerate(self.vocab)}

    def create_chunks(self):
        for idx in range(len(self.essay_id)):
            ess = self.text[idx]
            n = len(ess)
            self.new_data.append([ess[:n//3], self.scores[idx]])
            self.new_data.append([ess[n//3:2*n//3], self.scores[idx]])
            self.new_data.append([ess[2*n//3:], self.scores[idx]])
        self.new_data = np.array(self.new_data)

    def text_num(self):
        for i, line in enumerate(self.text):
            x = []
            for word in line:
                x.append(self.word_to_id[word])
            self.text[i] = x


import pandas as pd
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np


dataset = Dataset("/content/drive/My Drive/Colab Notebooks/NLP/data/training_data.csv")

dataset.preprocess()
dataset.create_vocab()
print(len(dataset.vocab))
print(dataset.vocab[:10])
print(dataset.word_to_id['!'])
dataset.create_chunks()
print(dataset.text[0])
"""
print(len(dataset.new_data), len(dataset.essay_id))
print("0: ", dataset.new_data[0, 0])
print("1: ", dataset.new_data[1, 0])
print("2: ", dataset.new_data[2, 0])
print(dataset.text[0])
"""

12527
['!', '&', "'", "''", "''cracked", "''from", "''i", "''laughter", "'d", "'em"]
0
['long', 'time', 'ago', 'third', 'grade', 'friend', "'s", 'mom', 'bad', 'mood', '.', 'never', 'laughed', 'never', 'smiled', '.', 'every', 'time', 'saw', 'would', 'smile', 'would', 'frown', 'keep', 'walking', '.', 'first', "n't", 'know', 'grouch', 'thought', "n't", 'like', 'something.when', 'told', 'mom', 'grouch', 'started', 'laugh', 'laugh', '.', 'asked', 'funny', 'told', 'thought', 'mom', "n't", 'like', 'something', 'every', 'time', 'see', 'mom', 'would', 'smile', 'frown', 'walk', 'away', '.', 'made', 'friend', 'laugh', 'cracking', 'hard', 'got', 'trouble', 'class', '.', 'next', 'day', 'eating', 'lunch', 'school', 'says', '&', 'lt', ';', 'hey', 'pretty', 'good', 'making', 'people', 'laugh', '&', 'gt', ';', '.', 'said', '&', 'lt', ';', 'jokesare', 'horrible', '&', 'gt', ';', '.', 'said', 'lets', 'put', 'test', 'go', 'one', 'new', 'school', '&', 'gt', ';', '.', 'said', 'went', 'around', 'whole', 'sch

'\nprint(len(dataset.new_data), len(dataset.essay_id))\nprint("0: ", dataset.new_data[0, 0])\nprint("1: ", dataset.new_data[1, 0])\nprint("2: ", dataset.new_data[2, 0])\nprint(dataset.text[0])\n'

In [0]:
dataset.text_num()
train_x = dataset.text

In [9]:
trainDF = trainDF[['text','organization']]
trainDF.columns = ['text','label']
trainDF.head()

,text,label
0,A long time ago when I was in third grade I h...,4
1,Softball has to be one of the single most gre...,4
2,"Some people like making people laugh, I love ...",3
3,"""LAUGHTER"" @CAPS1 I hang out with my friends...",3
4,Well ima tell a story about the time i got @CA...,2


In [10]:
trainDF.shape

(723, 2)

In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [12]:
max_len = 0
ind = 0
for t in train_x:
  if len(t.split()) > max_len:
    max_len = len(t.split())
max_len

856

In [0]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=max_len)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=max_len)

In [14]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
                vocabulary=None)

In [0]:
vocab_len = len(count_vect.vocabulary_)

In [16]:
print(train_seq_x.shape, train_y.shape)

(542, 856) (542,)


In [17]:
train_seq_x

array([[   0,    0,    0, ...,   10, 5416,   52],
       [   0,    0,    0, ...,  495,    4,  305],
       [   0,    0,    0, ..., 5893,    3,  375],
       ...,
       [   0,    0,    0, ...,    8,    5,   56],
       [   0,    0,    0, ...,   16,  166,   41],
       [   0,    0,    0, ..., 4706,  773, 5811]], dtype=int32)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

In [0]:
device = torch.device('cpu')
#'cuda:0' if torch.cuda.is_available() else 'cpu'  
train_seq_x1 = torch.from_numpy(train_seq_x).to(torch.int64).to(device)
train_y1 = torch.from_numpy(train_y).to(torch.int64).to(device)

valid_seq_x1 = torch.from_numpy(valid_seq_x).to(torch.int64).to(device)
valid_y1 = torch.from_numpy(valid_y).to(torch.int64).to(device)


batch_size = 50
train_loader = DataLoader(TensorDataset(train_seq_x1, train_y1), batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(TensorDataset(valid_seq_x1, valid_y1), batch_size = batch_size, shuffle = True)


In [52]:
train_seq_x1.shape, train_y1.shape

(torch.Size([542, 856]), torch.Size([542]))

In [53]:
device

device(type='cpu')

In [0]:

class LSTM_Model(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_num, output_num, layer_num):
      super().__init__()
      self.vocab_size = vocab_size
      self.layer_num = layer_num
      self.hidden_num = hidden_num

      self.embedding = nn.Embedding(vocab_size, embedding_size)
      self.lstm = nn.LSTM(embedding_size, hidden_num, layer_num)
      self.fc = nn.Linear(hidden_num, output_num)
      self.relu = nn.ReLU()

        
    def forward(self, word_seq):
      word_emb = self.embedding(word_seq)
      lstm_out,h = self.lstm(word_emb)
      lstm_out = lstm_out.contiguous().view(-1, self.hidden_num)
      fc_out = self.fc(lstm_out)
      relu_out = self.relu(fc_out)
      relu_out = relu_out.view(batch_size, -1) 
      relu_out = relu_out[:,-1]
      return relu_out, h


In [55]:
train_seq_x.shape

(542, 856)

In [0]:
def train(data_loader, classifier, loss_function, optimizer):
    classifier.train()
    loss = 0
    losses = []
    
    accuracy = 0
    accuracies = []
    for i, (texts, labels) in enumerate(data_loader):
      
        if(texts.shape[0] != batch_size):
            break
        labels = labels.float()
        # texts = texts.cuda()
        # labels = labels.cuda()
        optimizer.zero_grad()
        predictions,h = classifier(texts)
        print(predictions.type(), labels.type())
        loss = loss_function(predictions, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item()) 
        # print(losses)       
    return sum(losses)/len(losses)




In [0]:
n_vocab = vocab_len + 1000
# n_vocab = len(embedding_matrix)
n_embed = 300
n_hidden = 512
n_output = 1
n_layers = 2

rnn_model = LSTM_Model(n_vocab, n_embed, n_hidden, n_output, n_layers)
# rnn_model.cuda()
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(rnn_model.parameters(), lr=0.0001, momentum=0.9)
epochs = 5


In [0]:
for epoch in range(0, epochs):
    print("epoch:", epoch + 1)
    training_loss = train(train_loader, rnn_model, loss_function, optimizer)
    print("training_loss:", training_loss)


epoch: 1
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor
torch.FloatTensor torch.FloatTensor


In [0]:
train_seq_x.shape

In [0]:
train_seq_x[42]

In [0]:
train_y.shape

In [0]:
embedding_matrix.shape

In [0]:
embedding_matrix[226].shape